## 1. Install and Import Required Libraries

In [1]:
# Install required libraries
import subprocess
import sys

# Install transformers if not already installed
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers", "torch"])

print("✓ Libraries ready for import")

/home/shivang-patel/Internship/Kombee/Learning/Generative-AI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-27 12:21:27.851234: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-27 12:21:27.921054: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-27 12:21:30.718986: I tensorflow/core/util/port.cc:153] oneDNN custom operat

✓ Libraries ready for import


In [4]:
# Import required libraries
from transformers import pipeline, AutoTokenizer, AutoModel
import torch
import warnings

warnings.filterwarnings('ignore')

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cpu


## 2. Load a Pre-trained Model for Summarization

In [5]:
# Load summarization pipeline
print("Loading summarization model (BART)...")
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=0 if device == "cuda" else -1
)
print("✓ Summarization model loaded successfully!")

Loading summarization model (BART)...


Device set to use cpu


✓ Summarization model loaded successfully!


## 3. Perform Text Summarization

In [6]:
# Sample texts for summarization
sample_texts = {
    "Article 1": """Machine learning is a subset of artificial intelligence (AI) that provides systems 
    with the ability to automatically learn and improve from experience without being explicitly programmed. 
    Machine learning focuses on the development of computer programs that can access data and use it to learn for themselves. 
    The process of learning begins with observations or data, such as examples, direct experience, or instruction, 
    in order to look for patterns in data and make better decisions in the future based on the examples that we provide. 
    The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust actions accordingly.""",
    
    "Article 2": """The COVID-19 pandemic has caused unprecedented disruption to the global economy, healthcare systems, 
    and daily life. Lockdowns and social distancing measures forced businesses to close temporarily, leading to millions of job losses. 
    The travel and tourism industries were particularly hard hit, with airlines and hotels experiencing dramatic drops in demand. 
    Governments around the world implemented stimulus packages and relief programs to support workers and businesses. 
    Despite the challenges, the pandemic accelerated digital transformation across many sectors. 
    Remote work became mainstream, and many companies adapted their business models to survive and thrive in the new environment. 
    As vaccination rates increased, economies gradually reopened, though many sectors continued to face recovery challenges.""",
    
    "Article 3": """Climate change represents one of the most pressing challenges facing humanity today. 
    Rising global temperatures are causing more frequent extreme weather events, including hurricanes, floods, and droughts. 
    The melting of polar ice caps is contributing to rising sea levels, threatening coastal communities around the world. 
    Deforestation and industrial pollution continue to increase greenhouse gas concentrations in the atmosphere. 
    Scientists warn that without significant action to reduce emissions, the planet could face catastrophic consequences. 
    However, renewable energy technologies, electric vehicles, and sustainable agricultural practices offer promising solutions. 
    International agreements like the Paris Climate Accord represent global commitment to addressing this crisis through coordinated efforts."""
}

print("Sample texts loaded!")
for title in sample_texts:
    print(f"\n{title} ({len(sample_texts[title].split())} words)")

Sample texts loaded!

Article 1 (104 words)

Article 2 (110 words)

Article 3 (103 words)


In [7]:
# Function to summarize text
def summarize_text(text, max_length=150, min_length=50):
    """Summarize text with specified parameters"""
    summary = summarizer(
        text,
        max_length=max_length,
        min_length=min_length,
        do_sample=False
    )
    return summary[0]['summary_text']

# Summarize all sample texts
print("\n" + "="*80)
print("TEXT SUMMARIZATION RESULTS")
print("="*80)

for title, text in sample_texts.items():
    print(f"\n{title}")
    print("-" * 80)
    print(f"Original text ({len(text.split())} words):")
    print(f"{text[:200]}...\n")
    
    summary = summarize_text(text)
    print(f"Summary ({len(summary.split())} words):")
    print(summary)

Your max_length is set to 150, but your input_length is only 141. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)



TEXT SUMMARIZATION RESULTS

Article 1
--------------------------------------------------------------------------------
Original text (104 words):
Machine learning is a subset of artificial intelligence (AI) that provides systems 
    with the ability to automatically learn and improve from experience without being explicitly programmed. 
    Ma...

Summary (56 words):
Machine learning focuses on the development of computer programs that can access data and use it to learn for themselves. The process of learning begins with observations or data, such as examples, direct experience, or instruction. The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust actions accordingly.

Article 2
--------------------------------------------------------------------------------
Original text (110 words):
The COVID-19 pandemic has caused unprecedented disruption to the global economy, healthcare systems, 
    and daily life. Lockdowns and so

## 4. Load a Pre-trained Model for Question Answering

In [8]:
# Load question answering pipeline
print("Loading QA model (DistilBERT)...")
qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad",
    device=0 if device == "cuda" else -1
)
print("✓ Question Answering model loaded successfully!")

Loading QA model (DistilBERT)...


Device set to use cpu


✓ Question Answering model loaded successfully!


## 5. Perform Question Answering

In [9]:
# Context and questions for QA
qa_context = """Machine learning is a subset of artificial intelligence that enables systems to learn from data.
Deep learning models use neural networks with multiple layers to process information.
Natural language processing helps computers understand and process human language.
Transfer learning allows us to leverage knowledge from one task to improve another task.
The transformer architecture has revolutionized the field of NLP."""

qa_questions = [
    "What is machine learning?",
    "Which architecture revolutionized NLP?",
    "What does transfer learning allow?",
    "What are neural networks used for?"
]

print("Context for QA:")
print("-" * 80)
print(qa_context)
print("\nQuestions:")
for i, q in enumerate(qa_questions, 1):
    print(f"{i}. {q}")

Context for QA:
--------------------------------------------------------------------------------
Machine learning is a subset of artificial intelligence that enables systems to learn from data.
Deep learning models use neural networks with multiple layers to process information.
Natural language processing helps computers understand and process human language.
Transfer learning allows us to leverage knowledge from one task to improve another task.
The transformer architecture has revolutionized the field of NLP.

Questions:
1. What is machine learning?
2. Which architecture revolutionized NLP?
3. What does transfer learning allow?
4. What are neural networks used for?


In [10]:
# Function to answer questions
def answer_questions(context, questions):
    """Answer multiple questions from a context"""
    results = []
    for question in questions:
        result = qa_pipeline(question=question, context=context)
        results.append({
            'question': question,
            'answer': result['answer'],
            'score': result['score']
        })
    return results

# Get answers
qa_results = answer_questions(qa_context, qa_questions)

# Display results
print("\n" + "="*80)
print("QUESTION ANSWERING RESULTS")
print("="*80)

for i, result in enumerate(qa_results, 1):
    print(f"\nQ{i}: {result['question']}")
    print(f"Answer: {result['answer']}")
    print(f"Confidence: {result['score']:.4f}")
    print("-" * 80)


QUESTION ANSWERING RESULTS

Q1: What is machine learning?
Answer: a subset of artificial intelligence
Confidence: 0.3150
--------------------------------------------------------------------------------

Q2: Which architecture revolutionized NLP?
Answer: transformer
Confidence: 0.4510
--------------------------------------------------------------------------------

Q3: What does transfer learning allow?
Answer: leverage knowledge from one task to improve another task
Confidence: 0.4011
--------------------------------------------------------------------------------

Q4: What are neural networks used for?
Answer: process information
Confidence: 0.5087
--------------------------------------------------------------------------------


## 6. Compare Different Model Parameters

In [11]:
# Test different summarization parameters
test_text = sample_texts["Article 2"]

print("Testing different summarization parameters...\n")
print(f"Original text length: {len(test_text.split())} words\n")

parameters = [
    {"max_length": 50, "min_length": 20, "name": "Short Summary"},
    {"max_length": 150, "min_length": 50, "name": "Medium Summary"},
    {"max_length": 300, "min_length": 100, "name": "Long Summary"},
]

summaries_comparison = []
for param in parameters:
    summary = summarize_text(
        test_text,
        max_length=param["max_length"],
        min_length=param["min_length"]
    )
    summaries_comparison.append({
        'name': param['name'],
        'max_length': param['max_length'],
        'min_length': param['min_length'],
        'summary': summary,
        'word_count': len(summary.split())
    })

# Display comparison
for s in summaries_comparison:
    print(f"{s['name']} (max={s['max_length']}, min={s['min_length']}):")
    print(f"Generated: {s['word_count']} words")
    print(f"Summary: {s['summary']}\n")

Testing different summarization parameters...

Original text length: 110 words



Your max_length is set to 300, but your input_length is only 164. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=82)


Short Summary (max=50, min=20):
Generated: 36 words
Summary: The COVID-19 pandemic has caused unprecedented disruption to the global economy, healthcare systems, and daily life. Lockdowns and social distancing measures forced businesses to close temporarily, leading to millions of job losses. The travel and tourism

Medium Summary (max=150, min=50):
Generated: 50 words
Summary: The COVID-19 pandemic has caused unprecedented disruption to the global economy, healthcare systems, and daily life. Lockdowns and social distancing measures forced businesses to close temporarily, leading to millions of job losses. The travel and tourism industries were particularly hard hit, with airlines and hotels experiencing dramatic drops in demand.

Long Summary (max=300, min=100):
Generated: 80 words
Summary: The COVID-19 pandemic has caused unprecedented disruption to the global economy, healthcare systems, and daily life. Lockdowns and social distancing measures forced businesses to close temporarily,

In [12]:
# Test QA with different models and contexts
print("="*80)
print("ADVANCED QA EXAMPLES")
print("="*80)

# Example 1: Technical context
tech_context = """Python is a high-level programming language known for its simplicity and readability.
It supports multiple programming paradigms including procedural, object-oriented, and functional programming.
Python is widely used in data science, machine learning, web development, and automation.
The language includes a comprehensive standard library and a large ecosystem of third-party packages."""

tech_questions = [
    "What is Python?",
    "What programming paradigms does Python support?",
    "Where is Python commonly used?"
]

print("\n--- Technical Context ---")
print(f"Context: {tech_context[:100]}...\n")

tech_results = answer_questions(tech_context, tech_questions)
for result in tech_results:
    print(f"Q: {result['question']}")
    print(f"A: {result['answer']} (confidence: {result['score']:.4f})\n")

ADVANCED QA EXAMPLES

--- Technical Context ---
Context: Python is a high-level programming language known for its simplicity and readability.
It supports mu...

Q: What is Python?
A: a high-level programming language (confidence: 0.6153)

Q: What programming paradigms does Python support?
A: procedural, object-oriented, and functional programming (confidence: 0.5949)

Q: Where is Python commonly used?
A: data science, machine learning, web development, and automation (confidence: 0.5470)



In [13]:
# Example 2: Historical context
history_context = """The Renaissance was a period of European cultural, artistic, and intellectual revival,
spanning from the 14th to 17th centuries. It began in Italy and gradually spread throughout Europe.
Key figures included Leonardo da Vinci, Michelangelo, and Shakespeare.
The movement emphasized humanism, individualism, and the study of classical Greek and Roman texts.
The invention of the printing press by Johannes Gutenberg in 1440 played a crucial role in spreading Renaissance ideas."""

history_questions = [
    "What was the Renaissance?",
    "Who invented the printing press?",
    "When did the Renaissance span?",
    "Which artist was a key figure in the Renaissance?"
]

print("--- Historical Context ---")
print(f"Context: {history_context[:100]}...\n")

history_results = answer_questions(history_context, history_questions)
for result in history_results:
    print(f"Q: {result['question']}")
    print(f"A: {result['answer']} (confidence: {result['score']:.4f})\n")

--- Historical Context ---
Context: The Renaissance was a period of European cultural, artistic, and intellectual revival,
spanning from...

Q: What was the Renaissance?
A: a period of European cultural, artistic, and intellectual revival (confidence: 0.5865)

Q: Who invented the printing press?
A: Johannes Gutenberg (confidence: 0.9980)

Q: When did the Renaissance span?
A: 14th to 17th centuries (confidence: 0.4587)

Q: Which artist was a key figure in the Renaissance?
A: Leonardo da Vinci (confidence: 0.6384)



## 7. Interactive Demo

In [14]:
# Interactive summarization demo
def interactive_summarize():
    """Interactive summarization function"""
    print("\n" + "="*80)
    print("INTERACTIVE SUMMARIZATION DEMO")
    print("="*80 + "\n")
    
    # Use one of the predefined texts
    selected_text = sample_texts["Article 1"]
    print(f"Text to summarize (length: {len(selected_text.split())} words):")
    print(selected_text[:200] + "...\n")
    
    print("Generating summaries with different parameters...\n")
    
    configs = [
        {"max_length": 60, "min_length": 20},
        {"max_length": 100, "min_length": 50},
        {"max_length": 150, "min_length": 80},
    ]
    
    for i, config in enumerate(configs, 1):
        summary = summarize_text(
            selected_text,
            max_length=config["max_length"],
            min_length=config["min_length"]
        )
        print(f"Summary {i} (max={config['max_length']}, words={len(summary.split())}):")
        print(f"{summary}\n")

interactive_summarize()


INTERACTIVE SUMMARIZATION DEMO

Text to summarize (length: 104 words):
Machine learning is a subset of artificial intelligence (AI) that provides systems 
    with the ability to automatically learn and improve from experience without being explicitly programmed. 
    Ma...

Generating summaries with different parameters...

Summary 1 (max=60, words=40):
Machine learning focuses on the development of computer programs that can access data and use it to learn for themselves. The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust actions accordingly.



Your max_length is set to 150, but your input_length is only 141. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)


Summary 2 (max=100, words=56):
Machine learning focuses on the development of computer programs that can access data and use it to learn for themselves. The process of learning begins with observations or data, such as examples, direct experience, or instruction. The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust actions accordingly.

Summary 3 (max=150, words=78):
Machine learning focuses on the development of computer programs that can access data and use it to learn for themselves. The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust actions accordingly. The process of learning begins with observations or data, such as examples, direct experience, or instruction, in order to look for patterns in data and make better decisions in the future based on the examples that we provide.



In [15]:
# Interactive Q&A demo
def interactive_qa():
    """Interactive question answering function"""
    print("\n" + "="*80)
    print("INTERACTIVE QUESTION ANSWERING DEMO")
    print("="*80 + "\n")
    
    context = """Artificial intelligence (AI) is intelligence demonstrated by machines, 
    as opposed to natural intelligence displayed by animals and humans.
    Machine learning is a field of study that gives computers the ability to learn without being explicitly programmed.
    Deep learning is a subset of machine learning that uses neural networks with multiple layers.
    Natural Language Processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence 
    concerned with the interactions between computers and human language."""
    
    print(f"Context for QA:\n{context}\n")
    print("-" * 80 + "\n")
    
    questions = [
        "What is artificial intelligence?",
        "What is machine learning?",
        "What is NLP?",
        "What is deep learning?"
    ]
    
    results = answer_questions(context, questions)
    
    for i, result in enumerate(results, 1):
        print(f"Q{i}: {result['question']}")
        print(f"A{i}: {result['answer']}")
        print(f"Confidence Score: {result['score']:.4f}")
        print()

interactive_qa()


INTERACTIVE QUESTION ANSWERING DEMO

Context for QA:
Artificial intelligence (AI) is intelligence demonstrated by machines, 
    as opposed to natural intelligence displayed by animals and humans.
    Machine learning is a field of study that gives computers the ability to learn without being explicitly programmed.
    Deep learning is a subset of machine learning that uses neural networks with multiple layers.
    Natural Language Processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence 
    concerned with the interactions between computers and human language.

--------------------------------------------------------------------------------

Q1: What is artificial intelligence?
A1: interactions between computers and human language
Confidence Score: 0.3545

Q2: What is machine learning?
A2: a field of study that gives computers the ability to learn without being explicitly programmed
Confidence Score: 0.6077

Q3: What is NLP?
A3: Natural Language Pr